In [1]:
import pandas as pd
pd.options.plotting.backend='matplotlib'
# set cwd to 3 directory up
%pwd
%cd ../../../
%pwd



/root/Projects/rucio-ops/src


/root/Projects/rucio-ops/.venv/lib64/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/root/Projects/rucio-ops/src'

# SEAL dumps: Serialize into TimeRangeSourceData
---

In this step, we load the files that contain the dumps provided by SEAL and store them in a TimeRangeSourceData format. This format is a dictionary that contains the following keys:
start: the start time of the dump
end: the end time of the dump
file: the file that contains the dump
source: the source of the dump ( SEAL or Rucio ). In this case, it will be SEAL.

In [4]:
from infrastructure.repository.data_repository import list_files
from datetime import datetime
from core.entity import TimeRangeSourceData
dir = 'data/seal'
data_files = list_files(dir)
data_files = [x.split('.')[0] for x in data_files]
seal_dumps_time_ranges = []
for file in data_files:
    _, start_date, end_date = file.split('_')
    start_date = datetime.strptime(start_date, '%Y%m%d')
    end_date = datetime.strptime(end_date, '%Y%m%d')
    df = pd.read_csv(f"{dir}/{file}.csv")
    seal_dumps_time_ranges.append(TimeRangeSourceData(start=start_date, end=end_date, source='SEAL', file=f"{dir}/{file}.csv", df=df))


/tmp/ipykernel_648204/1826891564.py:12: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{dir}/{file}.csv")


## Check the continuity of the time ranges

For all the files that we have, we will check if the time intervals are chained. This means that the end time of a dump is the same as the start time of the next dump. If this is not the case, we will print an error message.

In [ ]:
# sort by start date
seal_dumps_time_ranges = sorted(seal_dumps_time_ranges, key=lambda x: x.start)

# check if the entrires form a continuous time range
for i in range(1, len(seal_dumps_time_ranges)):
    if seal_dumps_time_ranges[i].start != seal_dumps_time_ranges[i-1].end:
        print(f"Continuity Error: {seal_dumps_time_ranges[i-1].end} != {seal_dumps_time_ranges[i].start}")
        print(f"Check files {seal_dumps_time_ranges[i-1].file} and {seal_dumps_time_ranges[i].file}")
        


## Merge the data across different time ranges

### Shortlist the final set of time ranges that form a continuous chain.

In [4]:
selected_time_ranges = seal_dumps_time_ranges[0:]

### Concatenate the time ranges to form a single time range.

In [5]:
seal_dumps_concatenated_df = pd.concat([x.df for x in selected_time_ranges])

# sort by mtime, then by name
seal_dumps_concatenated_df = seal_dumps_concatenated_df.sort_values(by=['mtime', 'name'])

# convert mtime to datetime
seal_dumps_concatenated_df['mtime'] = pd.to_datetime(seal_dumps_concatenated_df['mtime'])

### Add a month and year key to each row

In [6]:
# add a month column
seal_dumps_concatenated_df['month'] = seal_dumps_concatenated_df['mtime'].dt.month

# add a year column
seal_dumps_concatenated_df['year'] = seal_dumps_concatenated_df['mtime'].dt.year


In [ ]:
len(seal_dumps_concatenated_df[(seal_dumps_concatenated_df['month'] == 5) & (seal_dumps_concatenated_df['year'] == 2022) ])
# seal_dumps_concatenated_df["year"].unique()


# Rucio Dumps: Serialize Rucio dumps into Pandas DataFrame
---

In this step, we load the files that contain the dumps provided by Rucio and store them in a Pandas Dataframe format.

In [8]:
rucio_dumps_file = 'data/rucio/SEAL_TEST_2023-04-10'
data = []
with open(rucio_dumps_file) as f:
    rucio_dumps_data = f.readlines()
    for rucio_dumps_row in rucio_dumps_data:
        columns = rucio_dumps_row.split('\t')
        columns = [x.strip() for x in columns]
        rse, scope, name, checksum, size, creation_date, path, update_date, state , _, _, _= columns
        __file_name = path.split('/')[-1]
        __start_path = '/'.join(path.split('/')[0:-3])
        path = f"{__start_path}/{__file_name}"
        if(path.startswith('/')):
            path = path[1:]
        
        size = int(size)
        creation_date = datetime.strptime(creation_date, '%Y-%m-%d %H:%M:%S')
        update_date = datetime.strptime(update_date, '%Y-%m-%d %H:%M:%S')
        
        data.append([rse, scope, name, checksum, size, creation_date, path, update_date, state])

rucio_dumps_df = pd.DataFrame(columns=['rse', 'scope', 'name', 'checksum', 'size', 'creation_date', 'path', 'update_date', 'state'], data=data)

# sort by creation date
rucio_dumps_df = rucio_dumps_df.sort_values(by=['creation_date'])


### Add a month and year key to each row of rucio dumps

In [9]:
rucio_dumps_df['month'] = rucio_dumps_df['creation_date'].dt.month
rucio_dumps_df['year'] = rucio_dumps_df['creation_date'].dt.year

## Lost DIDs

We will check if there are any files that are present in the Rucio dumps but not in the SEAL dumps. If there are any such files, these files are marked as lost files.

In [10]:
rucio_dumps_df.set_index('path')
seal_dumps_concatenated_df.set_index('path')

lost_dids = rucio_dumps_df[~rucio_dumps_df['path'].isin(seal_dumps_concatenated_df['path'])]

## Dark DIDs

We will check if there are any files that are present in the SEAL dumps but not in the Rucio dumps. If there are any such files, these files are marked as dark data.

In [11]:
dark_dids = seal_dumps_concatenated_df[~seal_dumps_concatenated_df['path'].isin(rucio_dumps_df['path'])]

# Statistics

In [ ]:
time_range_start = seal_dumps_time_ranges[0].start
time_range_end = seal_dumps_time_ranges[-1].end
print(f"This report is generated from {time_range_start} to {time_range_end}")

## Lost DIDs Analysis

In [ ]:
num_rucio_dids = rucio_dumps_df['name'].nunique()
percentage_lost_num_dids = round((lost_dids.shape[0] / num_rucio_dids) * 100, 2)

print(f"Number of Rucio DIDs: {num_rucio_dids}")
print(f"Number of lost files: {lost_dids.shape[0]}")
print(f"Percentage of Lost Files: {percentage_lost_num_dids}%")


We will invalidate the files reported as lost that do not fall in the time period under consideration.

In [ ]:
invalid_lost_files = lost_dids[(lost_dids['creation_date'] < time_range_start) | (lost_dids['creation_date'] > time_range_end)]

print(f"Number of invalid lost files: {invalid_lost_files.shape[0]}")

In [ ]:
from core.utils import bytesToTB
total_size_rucio = rucio_dumps_df['size'].sum()
size_lost_files = lost_dids['size'].sum()
percentage_lost_size = round((size_lost_files / total_size_rucio) * 100, 2)


print(f"Total size of Rucio DIDs: {bytesToTB(total_size_rucio)} TB")
print(f"Total size of Lost DIDs: {bytesToTB(size_lost_files)} TB")
print(f"Percentage of Lost Size: {percentage_lost_size}%")


In [ ]:
import matplotlib.pyplot as plt

labels = ['Available', 'Lost']
sizes = [total_size_rucio - size_lost_files, size_lost_files]

colors = ['olivedrab','saddlebrown']

fig, axs = plt.subplots(1, 2)

axs[0].pie([num_rucio_dids - lost_dids.shape[0], lost_dids.shape[0]], labels=['Available', 'Lost'], autopct='%1.1f%%', startangle=90, colors=colors)
axs[0].set_title('Lost Data ( by Number of DIDs )')
axs[0].set_xlabel(f'Total DIDs: {num_rucio_dids}')

axs[1].pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)
axs[1].set_title('Lost Data ( by Size )')
axs[1].set_xlabel(f'Total Size: {bytesToTB(total_size_rucio)} TB')

plt.show()

## Dark DIDs Analysis

In [ ]:
num_seal_dids = seal_dumps_concatenated_df['name'].nunique()
num_dark_dids = dark_dids['name'].nunique()
percentage_dark = round((dark_dids.shape[0] / num_seal_dids) * 100, 2)

print(f"Number of SEAL DIDs: {num_seal_dids}")
print(f"Number of Dark Files: {dark_dids.shape[0]}")
print(f"Percentage of Dark Data: {percentage_dark}%" )

In [ ]:
total_size_seal = seal_dumps_concatenated_df['size'].sum()
size_dark_files = dark_dids['size'].sum()
percentage_dark_size = round((size_dark_files / total_size_seal) * 100, 2)

print(f"Total size of SEAL DIDs: {bytesToTB(total_size_seal)} TB")
print(f"Total size of Dark Data: {bytesToTB(size_dark_files)} TB")
print(f"Percentage of Dark Size: {percentage_dark_size}%")

In [ ]:
fig, axs = plt.subplots(1, 2)

sizes = [total_size_seal - size_dark_files, size_dark_files]

axs[0].pie([num_seal_dids - dark_dids.shape[0], dark_dids.shape[0]], labels=['Available', 'Dark'], autopct='%1.1f%%', startangle=90, colors=colors)
axs[0].set_title('Dark Data ( by Number of DIDs )')
axs[0].set_xlabel(f'Total DIDs: {num_seal_dids}')

axs[1].pie(sizes, labels=["Available", "Dark"], autopct='%1.1f%%', startangle=90, colors=colors)
axs[1].set_title('Dark Data ( by Size )')
axs[1].set_xlabel(f'Total Size: {bytesToTB(total_size_seal)} TB')



plt.show()

# Consistent DIDs

Consistent DIDs are present in both Rucio and SEAL dumps.

In [ ]:
consistent_dids = rucio_dumps_df[rucio_dumps_df['path'].isin(seal_dumps_concatenated_df['path'])]

num_consistent_dids = consistent_dids['name'].nunique()
percentage_consistent = round((num_consistent_dids / num_rucio_dids) * 100, 2)

print(f"Number of Consistent DIDs: {num_consistent_dids}")
print(f"Percentage of Consistent DIDs: {percentage_consistent}%")

In [ ]:
size_consistent_files = consistent_dids['size'].sum()
percentage_consistent_size = round((size_consistent_files / total_size_rucio) * 100, 2)

print(f"Total size of Consistent DIDs: {bytesToTB(size_consistent_files)} TB")
print(f"Percentage of Consistent Size: {percentage_consistent_size}%")

# Save the results

In [22]:
start_date = time_range_start.strftime('%Y%m%d')
end_date = time_range_end.strftime('%Y%m%d')
lost_dids.to_csv(f'data/outputs/{start_date}-{end_date}_lost_dids.csv', index=False)
dark_dids.to_csv(f'data/outputs/{start_date}-{end_date}_dark_dids.csv', index=False)
consistent_dids.to_csv(f'data/outputs/{start_date}-{end_date}_consistent_dids.csv', index=False)

# Monthly Analysis

In [ ]:
print(time_range_start, time_range_end)

In [ ]:
# break into month, year tuples
lost_dids['month_year'] = list(zip(lost_dids['month'], lost_dids['year']))
dark_dids['month_year'] = list(zip(dark_dids['month'], dark_dids['year']))
consistent_dids['month_year'] = list(zip(consistent_dids['month'], consistent_dids['year']))

In [ ]:
lost_dids['month_year'].unique()
dark_dids['month_year'].unique()
consistent_dids['month_year'].unique()

In [26]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [27]:
pd.options.plotting.backend='plotly'


In [ ]:
fig = make_subplots(
    rows=len(lost_dids["month_year"].unique()), 
    cols=2,
    specs=[[{"type": "pie"}, {"type": "pie"}]] * len(lost_dids["month_year"].unique()),
    row_titles=[f'{month}/{year}' for month, year in lost_dids['month_year'].unique()],
    column_titles=['Count', 'Size']
)
monthly_stats_data = []

for index, month_year in enumerate(lost_dids['month_year'].unique()):
    month, year = month_year
    monthly_lost_dids = lost_dids[(lost_dids['month'] == month) & (lost_dids['year'] == year)]
    monthly_dark_dids = dark_dids[(dark_dids['month'] == month) & (dark_dids['year'] == year)]
    monthly_consistent_dids = consistent_dids[(consistent_dids['month'] == month) & (consistent_dids['year'] == year)]

    monthly_lost_dids_count = monthly_lost_dids['name'].nunique()
    monthly_dark_dids_count = monthly_dark_dids['name'].nunique()
    monthly_consistent_dids_count = monthly_consistent_dids['name'].nunique()
    monthly_total_count = monthly_lost_dids_count + monthly_dark_dids_count + monthly_consistent_dids_count

    monthly_lost_dids_size = monthly_lost_dids['size'].sum()
    monthly_dark_dids_size = monthly_dark_dids['size'].sum()
    monthly_consistent_dids_size = monthly_consistent_dids['size'].sum()
    monthly_total_size = monthly_lost_dids_size + monthly_dark_dids_size + monthly_consistent_dids_size

    monthly_df = pd.DataFrame(data={'Category': ['Lost', 'Dark', 'Consistent'], 'Count': [monthly_lost_dids_count, monthly_dark_dids_count, monthly_consistent_dids_count]})

    monthly_stats = [
        f"{month}/{year}",
        monthly_lost_dids_count,
        monthly_dark_dids_count,
        monthly_consistent_dids_count,
        monthly_total_count,
        monthly_lost_dids_size,
        monthly_dark_dids_size,
        monthly_consistent_dids_size,
        monthly_total_size
    ]
    monthly_stats_data.append(monthly_stats)

    fig.add_trace(
        go.Pie(
            textinfo='label+percent',
            titleposition="bottom center",
            labels=['Lost', 'Dark', 'Consistent'],
            values=[monthly_lost_dids_count, monthly_dark_dids_count, monthly_consistent_dids_count],
            title=f'Count: {monthly_total_count} DIDs',
            marker=dict(colors=['red', 'black', 'green'])
        ), 
    row=index+1,
    col=1
    )
    fig.add_trace(
        go.Pie(
            textinfo="label+percent",
            titleposition="bottom center",
            labels=['Lost', 'Dark', 'Consistent'],
            values=[monthly_lost_dids_size, monthly_dark_dids_size, monthly_consistent_dids_size],
            title=f'Size: {bytesToTB(monthly_total_size)} TB',
            marker=dict(colors=['red', 'black', 'green'])
        ), 
        row=index+1,
        col=2
    )

fig.update_layout(height=3000, width=1000, title_text=f"SEAL RSE: Monthly Consistency Report: {time_range_start} to {time_range_end}")
fig.show()

### Save the results of the monthly analysis

In [29]:
fig.write_html(f"reports/{start_date}-{end_date}_monthly_consistency_report.html")
fig.write_image(f"reports/{start_date}-{end_date}_monthly_consistency_report.png")

In [ ]:
# Make a line plot of the number of lost, dark and consistent DIDs over time using data frames in monthly_dfs
monthly_stats_df = pd.DataFrame(columns=['Month/Year', 'Lost Count', 'Dark Count', 'Consistent Count', 'Total Count', 'Lost Size', 'Dark Size', 'Consistent Size', 'Total Size'], data=monthly_stats_data)

fig = make_subplots(
    rows=3,
    cols=1,
    subplot_titles=('Lost (number of DIDs)', 'Dark (number of DIDs)', 'Consistent (number of DIDs)')
)
fig.add_trace(
    go.Scatter(x=monthly_stats_df['Month/Year'], y=monthly_stats_df['Lost Count'], mode='lines+markers', name='Lost DIDs'),
    row=1,
    col=1
)

fig.add_trace(
    go.Scatter(x=monthly_stats_df['Month/Year'], y=monthly_stats_df['Dark Count'], mode='lines+markers', name='Dark DIDs'),
    row=2,
    col=1
)

fig.add_trace(
    go.Scatter(x=monthly_stats_df['Month/Year'], y=monthly_stats_df['Consistent Count'], mode='lines+markers', name='Consistent DIDs'),
    row=3,
    col=1
)

fig.update_layout(height=1000, width=1000, title_text=f"SEAL RSE: Monthly Consistency Report: {time_range_start} to {time_range_end}")
fig.show()

fig.write_html(f"reports/{start_date}-{end_date}_monthly_stats_by_number_of_dids.html")
fig.write_image(f"reports/{start_date}-{end_date}_monthly_stats_by_number_of_dids.png")


In [ ]:
fig = make_subplots(
    rows=3,
    cols=1,
    subplot_titles=('Lost (size of DIDs)', 'Dark (size of DIDs)', 'Consistent (size of DIDs)')
)

fig.add_trace(
    go.Scatter(x=monthly_stats_df['Month/Year'], y=monthly_stats_df['Lost Size'], mode='lines+markers', name='Lost DIDs'),
    row=1,
    col=1
)

fig.add_trace(
    go.Scatter(x=monthly_stats_df['Month/Year'], y=monthly_stats_df['Dark Size'], mode='lines+markers', name='Dark DIDs'),
    row=2,
    col=1
)

fig.add_trace(
    go.Scatter(x=monthly_stats_df['Month/Year'], y=monthly_stats_df['Consistent Size'], mode='lines+markers', name='Consistent DIDs'),
    row=3,
    col=1
)

fig.update_layout(height=1000, width=1000, title_text=f"SEAL RSE: Monthly Consistency Report: {time_range_start} to {time_range_end}")
fig.show()

In [41]:
# save the monthly stats data frame

monthly_stats_df.to_csv(f'data/outputs/{start_date}-{end_date}_monthly_stats.csv', index=False)


# save the plots

fig.write_html(f"reports/{start_date}-{end_date}_monthly_stats_by_volume_report.html")
fig.write_image(f"reports/{start_date}-{end_date}_monthly_stats_by_volume_report.png")